In [107]:
# -*- encoding:urf-8 -*-
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table like and matrices
import pandas as pd
import numpy as np

# Modeling Helper
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import missingno as msno

# Configure visualization
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 8,6

train_path = '../input/train.csv'
test_path = '../input/test.csv'
hol_path = '../input/holiday.csv'
train = pd.read_csv(train_path,encoding='gbk',engine='python')
test = pd.read_csv(test_path)
holiday = pd.read_csv(hol_path)

In [108]:
train.rename(columns = {'BigCode':'parClass','MidCode':'Class'},inplace = True)
train

,custid,parClass,BigName,Class,MidName,SmallCode,SmallName,SaleDate,SaleMonth,Code,Standard,CumType,Unit,SaleNum,SalePrice,UnitPrice,Coupon
0,0,12,蔬果,1201,蔬菜,120109,其它蔬菜,20150101,201501,DW-1201090311,,生鲜,个,8.000,4.00,2.00,0
1,1,20,粮油,2014,酱菜类,201401,榨菜,20150101,201501,DW-2014010019,60g,一般商品,袋,6.000,3.00,0.50,0
2,2,15,日配,1505,冷藏乳品,150502,冷藏加味酸乳,20150101,201501,DW-1505020011,150g,一般商品,袋,1.000,2.40,2.40,0
3,3,15,日配,1503,冷藏料理,150305,冷藏面食类,20150101,201501,DW-1503050035,500g,一般商品,袋,1.000,6.50,8.30,0
4,4,15,日配,1505,冷藏乳品,150502,冷藏加味酸乳,20150101,201501,DW-1505020020,100g*8,一般商品,袋,1.000,11.90,11.90,0
5,5,30,洗化,3018,卫生巾,301802,夜用卫生巾,20150101,201501,DW-3018020109,10片,一般商品,包,1.000,8.90,8.90,0
6,6,12,蔬果,1201,蔬菜,120104,花果,20150101,201501,DW-1201040022,散称,生鲜,千克,0.964,8.07,5.60,0
7,7,20,粮油,2001,袋装速食面,200101,牛肉口味,20150101,201501,DW-2001010062,120g,一般商品,袋,1.000,2.50,3.00,0
8,8,13,熟食,1308,现制中式面点,130803,现制烙类,20150101,201501,DW-1308030035,个,生鲜,个,2.000,2.00,1.00,0
9,9,22,休闲,2203,膨化点心,220302,袋装薯片,20150101,201501,DW-2203020029,45g,一般商品,袋,1.000,4.00,4.00,0


### 重新整合train集

In [109]:
train.loc[:,'saleCount'] = 1
coord_class = train.groupby(['Class','SaleDate'],as_index=False)['saleCount'].sum()
coord_parClass = train.groupby(['parClass','SaleDate'],as_index=False)['saleCount'].sum()
coord_parClass.rename(columns = {'parClass':'Class'},inplace=True)
train_new = pd.concat([coord_class,coord_parClass],axis = 0)
train_new.loc[:,'parClass'] = train_new.Class.map(lambda x: str(x)[:2])
train_new.loc[:,'parClass'] = train_new.parClass.map(lambda x: int(x))
train_new

,Class,SaleDate,saleCount,parClass
0,1001,20150101,3,10
1,1001,20150102,6,10
2,1001,20150103,7,10
3,1001,20150104,5,10
4,1001,20150105,9,10
5,1001,20150106,5,10
6,1001,20150107,28,10
7,1001,20150108,7,10
8,1001,20150109,4,10
9,1001,20150110,4,10


### 节假日特征

In [110]:
train = pd.merge(train, holiday, on = 'SaleDate',how = 'left')
train_new = pd.merge(train_new, holiday, on = 'SaleDate',how = 'left')
test = pd.merge(test, holiday, on = 'SaleDate',how = 'left')

### 时间特征

In [111]:
def timeHandle(s):
    s = str(s)
    s = [s[:4],s[4:6],s[6:]]
    return '-'.join(s)

In [112]:
train.SaleDate = train.SaleDate.map(lambda x: timeHandle(x))
train.SaleDate = pd.to_datetime(train.SaleDate)
train.loc[:,'month'] = train.SaleDate.dt.month
train.loc[:,'dayOfWeek'] = train.SaleDate.dt.dayofweek
train.loc[:,'dayOfYear'] = train.SaleDate.dt.dayofyear
train.loc[:,'weekOfYear'] = train.SaleDate.dt.weekofyear

train_new.SaleDate = train_new.SaleDate.map(lambda x: timeHandle(x))
train_new.SaleDate = pd.to_datetime(train_new.SaleDate)
train_new.loc[:,'month'] = train_new.SaleDate.dt.month
train_new.loc[:,'dayOfWeek'] = train_new.SaleDate.dt.dayofweek
train_new.loc[:,'dayOfYear'] = train_new.SaleDate.dt.dayofyear
train_new.loc[:,'weekOfYear'] = train_new.SaleDate.dt.weekofyear

test.SaleDate = test.SaleDate.map(lambda x: timeHandle(x))
test.SaleDate = pd.to_datetime(test.SaleDate)
test.loc[:,'month'] = test.SaleDate.dt.month
test.loc[:,'dayOfWeek'] = test.SaleDate.dt.dayofweek
test.loc[:,'dayOfYear'] = test.SaleDate.dt.dayofyear
test.loc[:,'weekOfYear'] = test.SaleDate.dt.weekofyear

### 商品特征

#### 商品类别

In [113]:
cumDict = {u'一般商品':0.6089,u'生鲜':0.3782,'联营商品':0.0129}
train.CumType = train.CumType.map(cumDict)

In [114]:
midClassSet = set(train.Class)
bigClassSet = set(train.parClass)
midClassDict = {}
bigClassDict = {}
classDict = {}
for eachMid in midClassSet:
    coord = train[train.Class == eachMid].groupby('CumType')['Class'].count()
    sum = 0 
    for i in range(len(coord)):
        sum += coord.index[i] *  coord.values[i]
    rate = round(sum / (1.0 * len(coord) + 1),2) ##修正
    midClassDict[eachMid] = rate
    classDict[eachMid] = rate
for eachBig in bigClassSet:
    coord = train[train.parClass == eachBig].groupby('CumType')['Class'].count()
    sum = 0 
    for i in range(len(coord)):
        sum += coord.index[i] *  coord.values[i]
    rate = round(sum / (1.0 * len(coord) + 1),2) 
    bigClassDict[eachBig] = rate
    classDict[eachBig] = rate
train.loc[:,'cumType'] = train.Class.map(midClassDict)
train.loc[:,'parCumType'] = train.parClass.map(bigClassDict)

#最开始就需要改名
test.rename(columns={'Code':'Class'},inplace = True)
test.loc[:,'parClass'] = test.Class.map(lambda x: str(x)[:2])
test.loc[:,'parClass'] = test.parClass.map(lambda x: int(x))
test.loc[:,'cumType'] = test.Class.map(classDict)
test.loc[:,'parCumType'] = test.parClass.map(bigClassDict)

# 测试集中类cumType的缺失值处理,使用临近中类的type值
test = test.fillna('pad')

#### 商品热门指数

In [115]:
hotIndexDict = {}
parHotIndexDict = {}
totHotIndexDict = {}
totSaleCount = train.shape[0]
for eachMid in midClassSet:
    rate = round(train[train.Class == eachMid].shape[0] / (1.0 * totSaleCount),5)
    hotIndexDict[eachMid] = rate
    totHotIndexDict[eachMid] = rate
for eachBig in bigClassSet:
    rate = round(train[train.parClass == eachBig].shape[0] / (1.0 * totSaleCount),5)
    parHotIndexDict[eachBig] = rate
    totHotIndexDict[eachBig] = rate
    
train.loc[:,'hotIndex'] = train.Class.map(hotIndexDict)
train.loc[:,'parHotIndex'] = train.parClass.map(parHotIndexDict)

test.loc[:,'hotIndex'] = train.Class.map(totHotIndexDict)
test.loc[:,'parHotIndex'] = train.parClass.map(parHotIndexDict)
test

,Class,SaleDate,SaleNum,holidayCluster,disHoliday,month,dayOfWeek,dayOfYear,weekOfYear,parClass,cumType,parCumType,hotIndex,parHotIndex
0,1001,2015-05-01,1,3,1.0,5,4,121,18,10,175.86,157.19,0.23107,0.33148
1,1001,2015-05-02,1,3,1.0,5,5,122,18,10,175.86,157.19,0.01721,0.13720
2,1001,2015-05-03,1,3,1.0,5,6,123,18,10,175.86,157.19,0.02758,0.11165
3,1001,2015-05-04,1,1,0.5,5,0,124,19,10,175.86,157.19,0.00145,0.11165
4,1001,2015-05-05,1,1,0.3,5,1,125,19,10,175.86,157.19,0.02758,0.11165
5,1001,2015-05-06,1,1,0.3,5,2,126,19,10,175.86,157.19,0.01446,0.07166
6,1001,2015-05-07,1,1,0.0,5,3,127,19,10,175.86,157.19,0.23107,0.33148
7,1001,2015-05-08,1,1,0.0,5,4,128,19,10,175.86,157.19,0.00677,0.13720
8,1001,2015-05-09,1,2,0.0,5,5,129,19,10,175.86,157.19,0.01042,0.02076
9,1001,2015-05-10,1,2,0.0,5,6,130,19,10,175.86,157.19,0.03410,0.20668


#### 商品工作日/非工作日销量统计值 - 用train_new表

In [116]:
train_new

,Class,SaleDate,saleCount,parClass,holidayCluster,disHoliday,month,dayOfWeek,dayOfYear,weekOfYear
0,1001,2015-01-01,3,10,3,1.0,1,3,1,1
1,1001,2015-01-02,6,10,3,1.0,1,4,2,1
2,1001,2015-01-03,7,10,3,1.0,1,5,3,1
3,1001,2015-01-04,5,10,1,0.5,1,6,4,1
4,1001,2015-01-05,9,10,1,0.3,1,0,5,2
5,1001,2015-01-06,5,10,1,0.3,1,1,6,2
6,1001,2015-01-07,28,10,1,0.0,1,2,7,2
7,1001,2015-01-08,7,10,1,0.0,1,3,8,2
8,1001,2015-01-09,4,10,1,0.0,1,4,9,2
9,1001,2015-01-10,4,10,2,0.0,1,5,10,2


In [117]:
train_wk = train_new[train_new.holidayCluster == 1]
train_hol = train_new[train_new.holidayCluster != 1]

coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_median':'median'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_mean':'mean'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_max':'max'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_min':'min'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')

coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_median':'median'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_mean':'mean'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_max':'max'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_min':'min'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')

coord = train_hol.groupby('Class',as_index=False)['saleCount'].agg({'holSaleCount':'count'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index=False)['saleCount'].agg({'wkSaleCount':'count'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
train_new.loc[:,'wkHolRatio'] = train_new['wkSaleCount'] / (1.0 * train_new['holSaleCount'])

coord = train_new.groupby('Class',as_index=False)['wkHolRatio'].mean()
test = pd.merge(test, coord, on = 'Class', how='left')

del train_new['wkSaleCount'],train_new['holSaleCount']
del train_wk,train_hol

train_new

,Class,SaleDate,saleCount,parClass,holidayCluster,disHoliday,month,dayOfWeek,dayOfYear,weekOfYear,wkDaySaleCount_median,wkDaySaleCount_mean,wkDaySaleCount_max,wkDaySaleCount_min,holDaySaleCount_median,holDaySaleCount_mean,holDaySaleCount_max,holDaySaleCount_min,wkHolRatio
0,1001,2015-01-01,3,10,3,1.0,1,3,1,1,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
1,1001,2015-01-02,6,10,3,1.0,1,4,2,1,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
2,1001,2015-01-03,7,10,3,1.0,1,5,3,1,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
3,1001,2015-01-04,5,10,1,0.5,1,6,4,1,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
4,1001,2015-01-05,9,10,1,0.3,1,0,5,2,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
5,1001,2015-01-06,5,10,1,0.3,1,1,6,2,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
6,1001,2015-01-07,28,10,1,0.0,1,2,7,2,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
7,1001,2015-01-08,7,10,1,0.0,1,3,8,2,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
8,1001,2015-01-09,4,10,1,0.0,1,4,9,2,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684
9,1001,2015-01-10,4,10,2,0.0,1,5,10,2,8.0,9.546667,35.0,2.0,5.0,5.631579,16.0,1.0,1.973684


#### 价格特征

In [118]:
coord_class = train.groupby('Class',as_index = False)['UnitPrice'].agg({'price_mean':'mean'})
coord_par_class = train.groupby('parClass',as_index = False)['UnitPrice'].agg({'price_mean':'mean'})
coord_par_class.rename(columns = {'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class, coord_par_class],axis = 0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')

coord_class = train.groupby('Class',as_index = False)['UnitPrice'].agg({'price_median':'median'})
coord_par_class = train.groupby('parClass',as_index = False)['UnitPrice'].agg({'price_median':'median'})
coord_par_class.rename(columns = {'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class, coord_par_class],axis = 0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')

#### 促销特征

商品促销时销量与不促销时销量的比值 

In [119]:
coord_class_bonus_count = train[train['Coupon'] == 1].groupby('Class',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count = train[train['Coupon'] == 1].groupby('parClass',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_bonus_count,coord_parclass_bonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classBonusSaleCount'] = train_new['classBonusSaleCount'].fillna(0)
test['classBonusSaleCount'] = test['classBonusSaleCount'].fillna(0)

coord_class_notbonus_count = train[train['Coupon'] == 0].groupby('Class',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count = train[train['Coupon'] == 0].groupby('parClass',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_notbonus_count,coord_parclass_notbonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classNotBonusSaleCount'] = train_new['classNotBonusSaleCount'].fillna(1)
test['classNotBonusSaleCount'] = test['classNotBonusSaleCount'].fillna(1)

# 计算促销与非促销的比值
train_new.loc[:,'bonusRatio'] = np.round(train_new['classBonusSaleCount'] / (1.0 * train_new['classNotBonusSaleCount']),4)
del train_new['classBonusSaleCount'],train_new['classNotBonusSaleCount']
test.loc[:,'bonusRatio'] = np.round(test['classBonusSaleCount'] / (1.0 * test['classNotBonusSaleCount']),4)
del test['classBonusSaleCount'],test['classNotBonusSaleCount']

商品节假日时促销销量与不促销销量的比值

In [120]:
train_wk = train[train.holidayCluster == 1]
train_hol = train[train.holidayCluster != 1]

coord_class_bonus_count = train_hol[train_hol['Coupon'] == 1].groupby('Class',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count = train_hol[train_hol['Coupon'] == 1].groupby('parClass',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_bonus_count,coord_parclass_bonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classBonusSaleCount'] = train_new['classBonusSaleCount'].fillna(0)
test['classBonusSaleCount'] = test['classBonusSaleCount'].fillna(0)

coord_class_notbonus_count = train_hol[train_hol['Coupon'] == 0].groupby('Class',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count = train_hol[train_hol['Coupon'] == 0].groupby('parClass',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_notbonus_count,coord_parclass_notbonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classNotBonusSaleCount'] = train_new['classNotBonusSaleCount'].fillna(1)
test['classNotBonusSaleCount'] = test['classNotBonusSaleCount'].fillna(1)

# 计算促销与非促销的比值
train_new.loc[:,'bonusHolRatio'] = np.round(train_new['classBonusSaleCount'] / (1.0 * train_new['classNotBonusSaleCount']),4)
del train_new['classBonusSaleCount'],train_new['classNotBonusSaleCount']
test.loc[:,'bonusHolRatio'] = np.round(test['classBonusSaleCount'] / (1.0 * test['classNotBonusSaleCount']),4)
del test['classBonusSaleCount'],test['classNotBonusSaleCount']

商品非节假日时促销销量与不促销销量的比值

In [121]:
coord_class_bonus_count = train_wk[train_wk['Coupon'] == 1].groupby('Class',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count = train_wk[train_wk['Coupon'] == 1].groupby('parClass',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_bonus_count,coord_parclass_bonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classBonusSaleCount'] = train_new['classBonusSaleCount'].fillna(0)
test['classBonusSaleCount'] = test['classBonusSaleCount'].fillna(0)

coord_class_notbonus_count = train_wk[train_wk['Coupon'] == 0].groupby('Class',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count = train_wk[train_wk['Coupon'] == 0].groupby('parClass',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_notbonus_count,coord_parclass_notbonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classNotBonusSaleCount'] = train_new['classNotBonusSaleCount'].fillna(1)
test['classNotBonusSaleCount'] = test['classNotBonusSaleCount'].fillna(1)

# 计算促销与非促销的比值
train_new.loc[:,'bonusNotHolRatio'] = np.round(train_new['classBonusSaleCount'] / (1.0 * train_new['classNotBonusSaleCount']),4)
del train_new['classBonusSaleCount'],train_new['classNotBonusSaleCount']
test.loc[:,'bonusNotHolRatio'] = np.round(test['classBonusSaleCount'] / (1.0 * test['classNotBonusSaleCount']),4)
del test['classBonusSaleCount'],test['classNotBonusSaleCount']

del train_wk,train_hol

商品周几促销的比例

In [148]:
train_coupon = train[train.Coupon == 1]
coord = train_coupon.groupby(['Class','dayOfWeek'],as_index=False)['dayOfWeek'].agg({'dayOfWeekCount':'count'})
var = train_coupon.groupby(['Class'],as_index=False)['dayOfWeek'].agg({'classCouponCount':'count'})
coord = pd.merge(coord, var, on = 'Class',how='left' )
coord.loc[:,'bonusWeekProb'] = coord['dayOfWeekCount'] / np.round((1.0 * coord['classCouponCount']))
coord_c = coord.copy()

coord = train_coupon.groupby(['parClass','dayOfWeek'],as_index=False)['dayOfWeek'].agg({'dayOfWeekCount':'count'})
var = train_coupon.groupby(['parClass'],as_index=False)['dayOfWeek'].agg({'classCouponCount':'count'})
coord = pd.merge(coord, var, on = 'parClass',how='left' )
coord.loc[:,'bonusWeekProb'] = coord['dayOfWeekCount'] / np.round((1.0 * coord['classCouponCount']))
coord.rename(columns={'parClass':'Class'},inplace=True)
coord_pc = coord.copy()

coord = pd.concat([coord_c,coord_pc],axis=0)

train_new = pd.merge(train_new, coord[['Class','dayOfWeek','bonusWeekProb']], on = ['Class','dayOfWeek'],how='left')
train_new['bonusWeekProb'] = temp['bonusWeekProb'].fillna(0)

test = pd.merge(test, coord[['Class','dayOfWeek','bonusWeekProb']], on = ['Class','dayOfWeek'],how='left')
test['bonusWeekProb'] = temp['bonusWeekProb'].fillna(0)

In [152]:
test.columns.shape

(29L,)